In [1]:
from sklearn.cluster import KMeans #k-means
import numpy as np
import pandas as pd
import cv2
import os
from PIL import Image 
import scipy.misc
import imageio
import glob
import csv


try:
    os.remove("vectores_frames_x_video.csv")
    print("File Removed!")
except:
    print("El archivo no existe o no se removió")

File Removed!


In [2]:
def create_words_counter_per_sentence(trainFileName):
    train_data = pd.read_csv(trainFileName,delimiter=";")
    sentences_train = [sentence[:len(sentence) -1 ] for sentence in train_data.orth]
    sentences = sentences_train
    numberofWords = []
    for sentence in sentences:
        sentence = sentence.split()
        counter = 0
        for  word in sentence:   
            word = word.lower()
            if((word != "[silence]" and word != "[silence")):
#                 print (word,counter)
                counter+=1
        numberofWords.append(counter)
    del train_data
    del sentences
    del sentences_train
    return numberofWords

In [3]:
# para determinar la duración de los videos
def getLength(filename):
    v=cv2.VideoCapture(filename)
    v.set(cv2.CAP_PROP_POS_AVI_RATIO,1)
    segs = v.get(cv2.CAP_PROP_POS_MSEC)
    segs=segs/1000
    return segs
def duraciones_de_videos(path):
    duraciones=[]    
    for folder in os.listdir(str(path)):
        for file in os.listdir(path+folder):
            video=path+folder+"/"+file
            lg=getLength(video)
            duraciones.append(lg)
    return duraciones

In [4]:
def getMean_noZeros(lista):
    suma=0
    contador=0
    for i in lista:
        if(i!=0):
            contador=contador+1
            suma=suma+i
#             print("I = "+str(i)) 
    return (suma/contador)

In [5]:
# Dense Optical Flow in OpenCV y creo el vector de vectores
filename = "../../corpus/train.sentences.pronunciations.multi.translations.csv"
totalWords = create_words_counter_per_sentence(filename) 
videos="Files/videos_nuevos/"
duraciones = duraciones_de_videos(videos)

path='Files/videos_nuevos/'
array_vecs=[]
array_imgs=[]
array_movs=[]
# vector_de_frames=[]
# vector_de_frames=np.zeros(1)
vector_de_angulos=[]
vector_de_angulos_por_video=[]
posicion=0
contador=0
for folder in os.listdir(path):    
    for file in os.listdir(str(path+folder)):
        print("Nombre del vídeo " + file)
        cap = cv2.VideoCapture(str(path+folder+'/'+file))
        ret, frame1 = cap.read()
        prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        hsv = np.zeros_like(frame1)
        hsv[...,1] = 255

        while(1):
            ret, frame2 = cap.read()
            try:
                next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

                array_vecs.append(([ang]))

                hsv[...,0] = ang*180/np.pi/2
                vector_de_angulos=np.array(hsv[...,0]).flatten()
                angulo_promedio_por_frame=getMean_noZeros(vector_de_angulos)
                vector_de_angulos_por_video.append(angulo_promedio_por_frame)
#                 contador=contador+1
#                 print(contador)

                prvs = next
            except:
                break
        tamaño_vec=len(vector_de_angulos_por_video)
        print("Tamaño del vec de angulos por video = "+str(tamaño_vec))
        with open('vectores_frames_x_video.csv', 'a', newline='') as csvfile:
#                 filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
                filewriter = csv.writer(csvfile)
                for i in range(totalWords[posicion]):
                    comienzo=int((tamaño_vec/totalWords[posicion])*(i))
                    fin=int((tamaño_vec/totalWords[posicion])*(i+1))
                    filewriter.writerow(vector_de_angulos_por_video[comienzo:fin])
                    print("Tamaño del vector_cortado de ang = "+str((tamaño_vec/totalWords[posicion])))
        vector_de_angulos_por_video=[]        
#     vector_de_frames=[]
    
    posicion=posicion+1    
    ##las siguientes dos lineas son para detener la ejecución en el video que se determine en posición
    if(posicion==2): 
        break
cap.release()
cv2.destroyAllWindows()

Nombre del vídeo 001.avi
Tamaño del vec de angulos por video = 88
Tamaño del vector_cortado de ang = 29.333333333333332
Tamaño del vector_cortado de ang = 29.333333333333332
Tamaño del vector_cortado de ang = 29.333333333333332
Nombre del vídeo 002.avi
Tamaño del vec de angulos por video = 163
Tamaño del vector_cortado de ang = 32.6
Tamaño del vector_cortado de ang = 32.6
Tamaño del vector_cortado de ang = 32.6
Tamaño del vector_cortado de ang = 32.6
Tamaño del vector_cortado de ang = 32.6


In [6]:
with open('vectores_frames_x_video.csv', 'r') as f:
#     reader = csv.reader(f, delimiter=',', quotechar='|')
    reader = csv.reader(f)
    # read file row by row
    i=0
    for row in reader:
#         i=i+len(row)
        print(np.array(row).shape)
#         print(row)
# print(vector_de_ángulos_por_frame)
   

(29,)
(29,)
(30,)
(32,)
(33,)
(32,)
(33,)
(33,)


In [7]:
# totalWords
# duraciones
print("Frames del video 1 = "+str(30*duraciones[0])+"--- Frame por recorte = "+str(30*duraciones[0]/totalWords[0])+ "N Recortes = "+str(totalWords[0]))
print("Frames del video 2 = "+str(30*duraciones[1])+"--- Frame por recorte = "+str(30*duraciones[1]/totalWords[1])+ "N Recortes = "+str(totalWords[1]))
# print("Frames del video 3 = "+str(30*duraciones[2])+"--- Frame por recorte = "+str(30*duraciones[2]/totalWords[2]))
# print(str(30*duraciones[0]+30*duraciones[1]+30*duraciones[2]))
print(str(30*duraciones[0]+30*duraciones[1]))
print(str(95*3+96))

Frames del video 1 = 88.99999999999999--- Frame por recorte = 29.66666666666666N Recortes = 3
Frames del video 2 = 164.0--- Frame por recorte = 32.8N Recortes = 5
253.0
381


In [8]:
try:
    os.remove("zeros.csv")
    print("File Removed!")
except:
    print("El archivo no existe o no se removió")
    
with open('zeros.csv', 'a', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(np.zeros(10))
with open('zeros.csv', 'a', newline='') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            filewriter.writerow(np.ones(50))
with open('zeros.csv', 'a', newline='') as csvfile:
            filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
            filewriter.writerow(np.zeros(20))
with open('zeros.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',', quotechar='|')
    for row in reader:
#         print(np.array(row).shape)
        print(len(row))

File Removed!
10
50
20
